In [ ]:
from haystack import *
from haystack.document_stores import FAISSDocumentStore
from haystack.schema import Document
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers

In [4]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
text = open("./data.txt").read()
docs = [Document(content = text)]
document_store.write_documents(docs)

NameError: name 'FAISSDocumentStore' is not defined

In [20]:
retriever = DensePassageRetriever(document_store=document_store,
                                query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                )

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [21]:
document_store.update_embeddings(retriever)

Documents Processed: 10000 docs [00:00, 13531.77 docs/s]                                                               


In [25]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to log params: Changing param values is not allowed. Param with key='pred

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Failed to log params: Changing param values is not allowed. Param with key='processor' was already logged with value='TextSimilarityProcessor' for run ID='6f6e8c7fc2be456d9b6a41b67ee83605'. Attempted logging new value 'SquadProcessor'.
ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.


In [27]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [37]:
def answer_question(question):
    prediction = pipe.run(query = question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})
    print_answers(prediction, details="minimal")

In [38]:
answer_question("What are some applications of machine learning")

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.86s/ Batches]

[   {   'answer': 'studying the associations between the products that people '
                  'buy',
        'context': ' of the applications of machine learning is studying the '
                   'associations between the products that people buy. If a '
                   'person buys a product, he will be sho'},
    {   'answer': 'medical diagnosis, image processing, prediction, '
                  'classification, learning association, regression',
        'context': 'd industries. For example, medical diagnosis, image '
                   'processing, prediction, classification, learning '
                   'association, regression etc.\n'
                   '\n'
                   'The intelligent sys'},
    {   'answer': 'prediction systems',
        'context': ' past data on the loan.\n'
                   '\n'
                   'Machine learning can also be used in the prediction '
                   'systems. Considering the loan example, to compute the '
            